In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')

In [2]:
import numpy as np
import requests

In [3]:
requests.get("http://google.com")

<Response [200]>

In [4]:
from src.utils.DatabaseConnect import DatabaseConnect

# single machine setup (mongo is running here localy)
# "ip a" for ip address
MY_DATABASE_LINK = 'mongodb://10.22.50.212:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

DatabaseConnect.DB_LINK = MY_DATABASE_LINK

projectsCollection = DatabaseConnect.mini_database.projects()
usersCollection = DatabaseConnect.mini_database.users()
print(projectsCollection)

Collection(Database(MongoClient(host=['10.22.50.212:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [5]:
from src.utils.CacheAdapter import JSONAdapter
from src.utils.DatasetManager import ProjectsDatasetManager

In [6]:
def flatternData(data : dict[str, list]) -> np.array(dict):
    # takes in data in form of dict, where each key is a user id and each value is a list of that user's projects
    # returns just flat list of these projects 
    result = []

    for projectsArray in data.values():
        for project in projectsArray:
            result.append(project)

    return result

In [7]:
from random import random
from time import sleep, time
import json

In [8]:
# Validators are used to filter data by quality, 
# for example, I can take only those project, that has long description, readme file and many stars

def projectDataIsSufficient(projectData):
    # filters sufficient data (has description and one(or both) of topics or language)
    return (projectData and projectData["description"] and (len(projectData["topics"]) or projectData["language"]))


USERS_NUMBER_TO_SCAN = 10

def extractScannedUsers(data):
    return list(data.keys())

counter = 0

cacheFileName = "cache__28-03-2025__(sufficient)_{0}.json"

adapter = JSONAdapter(cacheFileName)

ProjectsDatasetManager.usersCollection = usersCollection
ProjectsDatasetManager.projectsCollection = projectsCollection
ProjectsDatasetManager.translatorServers = ["http://3.95.55.206:8000/", "http://18.212.28.203:8000/"]
manager = ProjectsDatasetManager(USERS_NUMBER_TO_SCAN, validate = projectDataIsSufficient, cacheAdapter = adapter)

#with open("/home/trukhinmaksim/src/logs/ignoreUsers.json", encoding = "utf-8") as file:
#    manager.ignoreUsers(json.load(file))

print(manager.ignoredUsers)

startPoint = time()

for i in range(3):
    manager.fromDB()
    manager.preprocess()

    #print(len(manager.data))
    scanned = extractScannedUsers(manager.data)
    manager.ignoreUsers(scanned)

    adapter.collectionName = cacheFileName.format(i)
    #print(adapter.collectionName)
    #adapter.save(manager.data)

    counter += len(flatternData(manager.data))

    manager.clearData()

    sleepTime = random() * 15
    print(f"Scanned {USERS_NUMBER_TO_SCAN} users. Sleeping {sleepTime}")
    sleep(sleepTime)

endPoint = time()
print(f"Total scanned: {counter} projects")
print(f"Time spent: {endPoint - startPoint} s")


{}
Scanning user: 0
Scanning user: 1
Scanning user: 2
Scanning user: 3
Scanning user: 4
Scanning user: 5
Scanning user: 6
Scanning user: 7
Scanning user: 8
Scanning user: 9
http://18.212.28.203:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://18.212.28.203:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://18.212.28.203:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://18.212.28.203:8000/
<Response [200]>
http://18.212.28.203:8000/
<Response [200]>
http://18.212.28.203:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.206:8000/
<Response [200]>
http://3.95.55.20

KeyboardInterrupt: 

In [ ]:
print(manager.translateText("你好", 3, True))

In [ ]:
"""
import json

with open("/home/trukhinmaksim/src/logs/ognoreUsers.json", "w", encoding = "utf-8") as file:
    json.dump([*manager.ignoredUsers], fp = file)
"""
#print()